In [1]:
from tensorflow.keras.utils import text_dataset_from_directory
from tensorflow.strings import regex_replace

In [2]:
def preprocess_data(directory):
    data = text_dataset_from_directory(directory)
    return data.map(lambda text, label: (regex_replace(text, '<br />', ' '), label))

In [3]:
train_data = preprocess_data('/Applications/ML projects/Movie Review Polarity/Dataset/train')
test_data = preprocess_data('/Applications/ML projects/Movie Review Polarity/Dataset/test')

Found 25000 files belonging to 2 classes.


2023-03-04 07:08:23.465234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 25000 files belonging to 2 classes.


In [4]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [10]:
def MODEL(train_texts, max_tokens=1000, max_len=100):
    model = Sequential()
    model.add(Input(shape=(1,), dtype="string"))
    vectorize_layer = TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_len)
    vectorize_layer.adapt(train_texts)
    model.add(vectorize_layer)
    model.add(Embedding(max_tokens + 1, 128))
    model.add(LSTM(64))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
train_texts = train_data.map(lambda text, label: text)

In [12]:
model = MODEL(train_texts)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 128)          128128    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 181,761
Trainable params: 181,761
Non-trainable params: 0
________________________________________________

In [13]:
model.fit(train_data, epochs=10, verbose=1)

Epoch 1/10
782/782 [==============================] - 28s 33ms/step - loss: 0.5534 - accuracy: 0.7113
Epoch 2/10
782/782 [==============================] - 27s 34ms/step - loss: 0.4445 - accuracy: 0.7955
Epoch 3/10
782/782 [==============================] - 27s 34ms/step - loss: 0.4079 - accuracy: 0.8146
Epoch 4/10
782/782 [==============================] - 27s 34ms/step - loss: 0.3914 - accuracy: 0.8217
Epoch 5/10
782/782 [==============================] - 27s 34ms/step - loss: 0.3687 - accuracy: 0.8359
Epoch 6/10
782/782 [==============================] - 26s 34ms/step - loss: 0.3451 - accuracy: 0.8501
Epoch 7/10
782/782 [==============================] - 26s 34ms/step - loss: 0.3304 - accuracy: 0.8573
Epoch 8/10
782/782 [==============================] - 26s 33ms/step - loss: 0.3130 - accuracy: 0.8644
Epoch 9/10
782/782 [==============================] - 26s 34ms/step - loss: 0.2994 - accuracy: 0.8736
Epoch 10/10
782/782 [==============================] - 27s 34ms/step - loss: 0.282

In [14]:
model.fit(train_data, epochs=5, verbose=1)

Epoch 1/5
782/782 [==============================] - 26s 34ms/step - loss: 0.2679 - accuracy: 0.8880
Epoch 2/5
782/782 [==============================] - 26s 34ms/step - loss: 0.2655 - accuracy: 0.8881
Epoch 3/5
782/782 [==============================] - 26s 33ms/step - loss: 0.2601 - accuracy: 0.8925
Epoch 4/5
782/782 [==============================] - 26s 34ms/step - loss: 0.2474 - accuracy: 0.8981
Epoch 5/5
782/782 [==============================] - 27s 35ms/step - loss: 0.2370 - accuracy: 0.9016


In [23]:
model.evaluate(test_data)

782/782 [==============================] - 9s 11ms/step - loss: 0.6391 - accuracy: 0.7664


[0.6390857696533203, 0.7664399743080139]